In [1]:
import pandas as pd

data = pd.read_csv("../data/data_official.csv")

In [2]:
cols = ['YEAR','CONF','QUAD ID','TEAM','SEED','ROUND','Wins','KADJ T', 'KADJ O', 'KADJ D', 'KADJ EM', 'EFG%','EFG%D','FTR','FTRD','TOV%','TOV%D','OREB%','DREB%','2PT%','2PT%D','3PT%','3PT%D','AST%','2PTR','3PTR','2PTRD','3PTRD','EFF HGT','EXP','TALENT','FT%','ELITE SOS','WAB']
df = data[cols]

In [3]:
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 6), "ROUND"] = 1
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 5), "ROUND"] = 2
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 4), "ROUND"] = 4
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 3), "ROUND"] = 8
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 2), "ROUND"] = 16
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 1), "ROUND"] = 32
df.loc[(df["YEAR"] == 2025) & (df["Wins"] == 0), "ROUND"] = 64

df.loc[(df["YEAR"] == 2025) & (df["TEAM"] == "San Diego St."), "ROUND"] = 68
df.loc[(df["YEAR"] == 2025) & (df["TEAM"] == "Saint Francis"), "ROUND"] = 68
df.loc[(df["YEAR"] == 2025) & (df["TEAM"] == "American"), "ROUND"] = 68
df.loc[(df["YEAR"] == 2025) & (df["TEAM"] == "Texas"), "ROUND"] = 68

In [4]:
years = list(df["YEAR"].unique())
quads = list(range(1,5))
seeds = list(range(1,17))

In [5]:
def combine_games(data):
    df = data.copy()
    years = list(range(2008, 2026))
    years.remove(2020)
    quads = list(range(1,5))
    
    cols = ['KADJ T', 'KADJ O', 'KADJ D', 'KADJ EM', 'EFG%','EFG%D','FTR','FTRD','TOV%','TOV%D','OREB%','DREB%','2PT%','2PT%D','3PT%','3PT%D','AST%','2PTR','3PTR','2PTRD','3PTRD','EFF HGT','EXP','TALENT','FT%','ELITE SOS','WAB']
    
    dfs = []
    
    for year in years:
        for quad in quads:
            dfx = df[(df["YEAR"] == year) & (df["QUAD ID"] == quad)]
            dfz = region_comb(dfx, year, cols)
            dfs.append(dfz)
            
        ### FINAL 4    
        dfk = df[(df["ROUND"] < 8) & (df["YEAR"] == year)]
        dfl = final4(dfk, year, cols)
        dfs.append(dfl)
        
        ### CHAMPIONSHIP
        dfq = df[(df["ROUND"] < 4) & (df["YEAR"] == year)]
        dfw = cg(dfq, year, cols)
        dfs.append(dfw)
            
    all_regions = pd.concat(dfs, axis=0)
    
    return all_regions

In [6]:
def region_comb(df, year, cols):
    seeds = list(range(1,17))
    
    dfs = []
    
    ### FIRST FOUR
    for seed in seeds:
        dfn = df[df["SEED"] == seed]
        if len(dfn) > 1:
            dfm = first4(dfn, year, cols)
            dfs.append(dfm)
    
    ### ROUND OF 64
    dfx = df[df["ROUND"] < 68]
    dfz = r64(dfx, year, cols)
    dfs.append(dfz)
    
    ### ROUND OF 32
    dff = df[df["ROUND"] < 64]
    dfg = r32(dff, year, cols)
    dfs.append(dfg)
    
    ### SWEET 16
    dfo = df[df["ROUND"] < 32]
    dfp = s16(dfo, year, cols)
    dfs.append(dfp)
    
    ### ELITE 8
    dfa = df[df["ROUND"] < 16]
    dfb = e8(dfa, year, cols)
    dfs.append(dfb)
           
    final_region = pd.concat(dfs, axis=0, ignore_index=True)
    return final_region

In [7]:
def first4(df, year, cols):
    team0 = df.head(1).reset_index(drop=True)
    team1 = df.tail(1).reset_index(drop=True)
    game = pd.DataFrame()
    for col in cols:
        game[col] = team0[col] - team1[col]
    game.insert(0, "YEAR", year)
    game.insert(1, "GAME_ROUND", "FIRST FOUR")
    game.insert(2, "Team0", team0["TEAM"])
    game.insert(3, "Team1", team1["TEAM"])
    game["Winner"] = 1 if team0["ROUND"].iloc[0] == 68 else 0
    
    return game

In [8]:
def r64(df, year, cols):
    seeds1 = [1,2,3,4,5,6,7,8]
    seeds2 = [16,15,14,13,12,11,10,9]
    
    games = []
    
    for i in range(8):
        team0 = df[df["SEED"] == seeds1[i]].reset_index(drop=True)
        team1 = df[df["SEED"] == seeds2[i]].reset_index(drop=True)
        game = pd.DataFrame()
        for col in cols:
            game[col] = team0[col] - team1[col]
        game.insert(0, "YEAR", year)
        game.insert(1, "GAME_ROUND", "ROUND OF 64")
        game.insert(2, "Team0", team0["TEAM"])
        game.insert(3, "Team1", team1["TEAM"])
        game["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
        games.append(game)
    
    return pd.concat(games, ignore_index=True)

In [9]:
def r32(df, year, cols):
    games = []
    
    team0 = df[(df["SEED"] == 1) | (df["SEED"] == 16)].reset_index(drop=True)
    team1 = df[(df["SEED"] == 8) | (df["SEED"] == 9)].reset_index(drop=True)
    game1 = pd.DataFrame()
    for col in cols:
        game1[col] = team0[col] - team1[col]
    game1.insert(0, "YEAR", year)
    game1.insert(1, "GAME_ROUND", "ROUND OF 32")
    game1.insert(2, "Team0", team0["TEAM"])
    game1.insert(3, "Team1", team1["TEAM"])
    game1["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game1)
    
    
    team0 = df[(df["SEED"] == 5) | (df["SEED"] == 12)].reset_index(drop=True)
    team1 = df[(df["SEED"] == 4) | (df["SEED"] == 13)].reset_index(drop=True)
    game2 = pd.DataFrame()
    for col in cols:
        game2[col] = team0[col] - team1[col]
    game2.insert(0, "YEAR", year)
    game2.insert(1, "GAME_ROUND", "ROUND OF 32")
    game2.insert(2, "Team0", team0["TEAM"])
    game2.insert(3, "Team1", team1["TEAM"])
    game2["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game2)
    
    team0 = df[(df["SEED"] == 6) | (df["SEED"] == 11)].reset_index(drop=True)
    team1 = df[(df["SEED"] == 3) | (df["SEED"] == 14)].reset_index(drop=True)
    game3 = pd.DataFrame()
    for col in cols:
        game3[col] = team0[col] - team1[col]
    game3.insert(0, "YEAR", year)
    game3.insert(1, "GAME_ROUND", "ROUND OF 32")
    game3.insert(2, "Team0", team0["TEAM"])
    game3.insert(3, "Team1", team1["TEAM"])
    game3["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game3)
    
    team0 = df[(df["SEED"] == 7) | (df["SEED"] == 10)].reset_index(drop=True)
    team1 = df[(df["SEED"] == 2) | (df["SEED"] == 15)].reset_index(drop=True)
    game4 = pd.DataFrame()
    for col in cols:
        game4[col] = team0[col] - team1[col]
    game4.insert(0, "YEAR", year)
    game4.insert(1, "GAME_ROUND", "ROUND OF 32")
    game4.insert(2, "Team0", team0["TEAM"])
    game4.insert(3, "Team1", team1["TEAM"])
    game4["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game4)
    
    return pd.concat(games, ignore_index=True)

In [10]:
def s16(df, year, cols):
    games = []
    
    team0 = df[(df["SEED"] == 1) | (df["SEED"] == 16) | (df["SEED"] == 8) | (df["SEED"] == 9)].reset_index(drop=True)
    team1 = df[(df["SEED"] == 5) | (df["SEED"] == 12) | (df["SEED"] == 4) | (df["SEED"] == 13)].reset_index(drop=True)
    game1 = pd.DataFrame()
    for col in cols:
        game1[col] = team0[col] - team1[col]
    game1.insert(0, "YEAR", year)
    game1.insert(1, "GAME_ROUND", "SWEET 16")
    game1.insert(2, "Team0", team0["TEAM"])
    game1.insert(3, "Team1", team1["TEAM"])
    game1["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game1)
    
    team0 = df[(df["SEED"] == 6) | (df["SEED"] == 11) | (df["SEED"] == 3) | (df["SEED"] == 14)].reset_index(drop=True)
    team1 = df[(df["SEED"] == 7) | (df["SEED"] == 10) | (df["SEED"] == 2) | (df["SEED"] == 15)].reset_index(drop=True)
    game2 = pd.DataFrame()
    for col in cols:
        game2[col] = team0[col] - team1[col]
    game2.insert(0, "YEAR", year)
    game2.insert(1, "GAME_ROUND", "SWEET 16")
    game2.insert(2, "Team0", team0["TEAM"])
    game2.insert(3, "Team1", team1["TEAM"])
    game2["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game2)
    
    return pd.concat(games, ignore_index=True)

In [11]:
def e8(df, year, cols):
    team0 = df.head(1).reset_index(drop=True)
    team1 = df.tail(1).reset_index(drop=True)
    game = pd.DataFrame()
    for col in cols:
        game[col] = team0[col] - team1[col]
    game.insert(0, "YEAR", year)
    game.insert(1, "GAME_ROUND", "ELITE 8")
    game.insert(2, "Team0", team0["TEAM"])
    game.insert(3, "Team1", team1["TEAM"])
    game["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    
    return game

In [12]:
def final4(df, year, cols):
    games = []
    
    team0 = df[(df["QUAD ID"] == 1) | (df["QUAD ID"] == 4)].reset_index(drop=True)
    team1 = df[(df["QUAD ID"] == 2) | (df["QUAD ID"] == 3)].reset_index(drop=True)
    game1 = pd.DataFrame()
    for col in cols:
        game1[col] = team0[col] - team1[col]
    game1.insert(0, "YEAR", year)
    game1.insert(1, "GAME_ROUND", "FINAL 4")
    game1.insert(2, "Team0", team0["TEAM"])
    game1.insert(3, "Team1", team1["TEAM"])
    game1["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    games.append(game1)
    
    return pd.concat(games, ignore_index=True)

In [13]:
def cg(df, year, cols):
    team0 = df.head(1).reset_index(drop=True)
    team1 = df.tail(1).reset_index(drop=True)
    game = pd.DataFrame()
    for col in cols:
        game[col] = team0[col] - team1[col]
    game.insert(0, "YEAR", year)
    game.insert(1, "GAME_ROUND", "CHAMPIONSHIP")
    game.insert(2, "Team0", team0["TEAM"])
    game.insert(3, "Team1", team1["TEAM"])
    game["Winner"] = 0 if team0["ROUND"].iloc[0] < team1["ROUND"].iloc[0] else 1
    
    return game

In [14]:
x = combine_games(df)

In [15]:
mm_games = x[['YEAR', 'GAME_ROUND', 'Team0', 'Team1', 'KADJ T', 'KADJ O', 'KADJ D',
       'KADJ EM', 'EFG%', 'EFG%D', 'FTR', 'FTRD', 'TOV%', 'TOV%D', 'OREB%',
       'DREB%', '2PT%', '2PT%D', '3PT%', '3PT%D', 'AST%', '2PTR', '3PTR',
       '2PTRD', '3PTRD', 'EFF HGT', 'EXP', 'TALENT', 'FT%', 'ELITE SOS', 'WAB',
       'Winner']].sort_values(["YEAR", "GAME_ROUND"], ascending=False)

In [16]:
round_order = ["FIRST FOUR", "ROUND OF 64", "ROUND OF 32", "SWEET 16", "ELITE 8", "FINAL 4", "CHAMPIONSHIP"]

# Convert GAME_ROUND to categorical with this order
mm_games["GAME_ROUND"] = pd.Categorical(mm_games["GAME_ROUND"], categories=round_order, ordered=True)

# Now sort by it
mm_games = mm_games.sort_values(["YEAR", "GAME_ROUND"], ascending=False)

In [17]:
mm_games.to_csv("../data/mm_games_diff.csv")